In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup, CData
import requests

from time import sleep
import pandas as pd 
import numpy as np

In [278]:
driver = webdriver.Chrome("/Applications/chromedriver")

In [96]:
# get main shelter that sit on top of parent shelters => Main > Parent > Child

driver.get("https://www.carrefour.fr/r?noRedirect=0")

page = driver.find_element_by_class_name('search-sidenav-item__list')
parent_urls = [elem.get_attribute("href") for elem in page.find_elements_by_tag_name('a')]
parent_urls = parent_urls[3:14]

urls = []

for children in parent_urls:
    driver.get(children)
    page = driver.find_element_by_class_name('search-sidenav-item--opened').find_element_by_class_name('search-sidenav-item__list')
    children_urls = [elem.get_attribute("href") for elem in page.find_elements_by_tag_name('a')]

    for child in children_urls:
        driver.get(child)
        try:
            page = driver.find_element_by_class_name('search-sidenav-item--opened').find_element_by_class_name('search-sidenav-item__list')
            a = [elem.get_attribute("href") for elem in page.find_elements_by_tag_name('a')]
            for url in a:
                urls.append(url)
        except:
            next
        
        
urls

['https://www.carrefour.fr/r/fruits-et-legumes/fruits/fruits-de-saison',
 'https://www.carrefour.fr/r/fruits-et-legumes/fruits/agrumes',
 'https://www.carrefour.fr/r/fruits-et-legumes/fruits/pommes-poires-et-raisins',
 'https://www.carrefour.fr/r/fruits-et-legumes/fruits/bananes-et-kiwis',
 'https://www.carrefour.fr/r/fruits-et-legumes/fruits/fruits-exotiques',
 'https://www.carrefour.fr/r/fruits-et-legumes/fruits/fruits-rouges',
 'https://www.carrefour.fr/r/fruits-et-legumes/fruits/melons-et-pasteques',
 'https://www.carrefour.fr/r/fruits-et-legumes/fruits/prunes-peches-nectarines-abricots',
 'https://www.carrefour.fr/r/fruits-et-legumes/legumes/legumes-de-saison',
 'https://www.carrefour.fr/r/fruits-et-legumes/legumes/choux-poireaux-navets-et-celeris',
 'https://www.carrefour.fr/r/fruits-et-legumes/legumes/carottes-et-pommes-de-terre',
 'https://www.carrefour.fr/r/fruits-et-legumes/legumes/salades-et-tomates',
 'https://www.carrefour.fr/r/fruits-et-legumes/legumes/concombres-avocats-

In [279]:
product_pages = []

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36'}

for link in urls:
    
    driver.get(link)

    sleep(1)
    shelter_main = driver.find_element_by_class_name('breadcrumb-trail__list').find_elements_by_css_selector('li')[-3].text
    shelter_parent = driver.find_element_by_class_name('breadcrumb-trail__list').find_elements_by_css_selector('li')[-2].text
    shelter_child = driver.find_element_by_class_name('breadcrumb-trail__list').find_elements_by_css_selector('li')[-1].text

    try:
        actions = ActionChains(driver)
        actions.move_to_element(driver.find_elements_by_css_selector("article")[-1]).perform()
    except:
        continue

    request_text = requests.get(link, headers=headers)   
    soup = BeautifulSoup(request_text.text,"html.parser" ,from_encoding="utf-8")  

    brands = []
    try:
        for label in driver.find_element_by_id('data_facet_marque').find_elements_by_css_selector("label"):
            brands.append(label.find_element_by_css_selector("span").text)
    except:
        continue

    product_pages.append({"url":link, 
                             "shelter_main": shelter_main,
                             "shelter_parent": shelter_parent,
                             "shelter_child": shelter_child,
                             "brands":brands,
                             "soup": soup.select('article')
                            })



JavascriptException: Message: javascript error: Cannot read property 'left' of undefined
  (Session info: chrome=78.0.3904.70)


In [280]:
data = pd.DataFrame(product_pages)
#data.to_csv('carrefour_catalog.csv')


In [284]:
pages = []
for index, row in data.iterrows():
    soup = row.soup
    pages.append({"shelter_main": row.shelter_main,
                  "shelter_parent": row.shelter_parent,
                  "shelter_child": row.shelter_child,                  
                  "brands": row.brands,
                  "page": soup
                })

In [331]:
products = []
for page in pages:
    for content in page['page'][1:-1]:
        name = content.find("a", class_="product-card-title--food")["title"]
        packaging = content.find("div", class_="product-card-packaging").text.strip()
        price = content.find("span", class_="product-card-price__price--final").text.strip()
        price_per_unit = content.find("div", class_="product-card-per-unit-label").text.strip()
#           promo_price = line[14:-1]
        image_url = content.find("img", class_="product-card-image__image")["src"]
        origin = content.find("div", class_="product-card__origin").text.strip() if content.find("div", class_="product-card__origin") else ""
        url = content.find("a", class_="product-card-title--food")["href"]
        product_id = content.find("div", class_="add-to-shoppinglist")["ean"]
        offer = content.find("div", class_="ds-product-card--vertical-highlightlabel").text.strip()
        offer_description = content.find("span", class_="promotion-label__txt").text.strip() if content.find("span", class_="promotion-label__txt") else ""
        if len(page['brands']) > 0:
            brand = ''.join([brand for brand in page['brands'] if brand in name])
        else:
            brand = ""
                
        products.append({"shelter_main": page['shelter_main'],
                         "shelter_parent": page['shelter_parent'],
                         "shelter_child": page['shelter_child'],  
                         "name":name, 
                         "packaging":packaging,                          
                         "brand": brand,
                         "price": price,
                         "price_per_unit":price_per_unit,
                        # "promo_price": promo_price,
                         "origin": origin,
                         "image_url": image_url,                    
                         "url": url,
                         "product_id": product_id,
                         "offer": offer,
                         "offer_description": offer_description
                        })

In [332]:
product_data = pd.DataFrame(products)

export_csv = product_data.to_csv (r'/Users/vincentsalamand/Downloads/products_carrefour.csv', index = None, header=True)


In [334]:
product_data.tail()

,brand,image_url,name,offer,offer_description,origin,packaging,price,price_per_unit,product_id,shelter_child,shelter_main,shelter_parent,url
20352,FORCE BIO,/images/loader-dots.svg,Jus de raisin 100% pur jus FORCE BIO,,,,la brique de 1 l,"4,05€",4.05 € / Litre,3760186970036,Jus de pommes et raisins,Boissons sans alcool,Jus de fruits et de légumes,/p/jus-de-raisin-100-pur-jus-force-bio-3760186...
20353,,/images/loader-dots.svg,Jus de raisin casher sans sucres ajoutés YARDEN,,,,la bouteille de 1 l,"2,50€",2.50 € / Litre,3423990000251,Jus de pommes et raisins,Boissons sans alcool,Jus de fruits et de légumes,/p/jus-de-raisin-casher-sans-sucres-ajoutes-ya...
20354,SOUS LE POMMIER,/images/loader-dots.svg,Jus pétillant de pomme non traitées SOUS LE PO...,,,,la bouteille de 75cL,"3,20€",4.27 € / Litre,3271471000661,Jus de pommes et raisins,Boissons sans alcool,Jus de fruits et de légumes,/p/jus-petillant-de-pomme-non-traitees-sous-le...
20355,,/images/loader-dots.svg,Jus pomme sans sucres ajoutés JOKER,Prix imbattable !,,,la brique d'1L,"1,29€",1.29 € / Litre,3123349014761,Jus de pommes et raisins,Boissons sans alcool,Jus de fruits et de légumes,/p/jus-pomme-sans-sucres-ajoutes-joker-3123349...
20356,,/images/loader-dots.svg,Jus de pomme du Val de Loire sans sucres ajout...,,,,la bouteille de 1l,"2,91€",2.91 € / Litre,3297760081005,Jus de pommes et raisins,Boissons sans alcool,Jus de fruits et de légumes,/p/jus-de-pomme-du-val-de-loire-sans-sucres-aj...
